In [1]:
# task 4.1. Реализовать BiRNN & LSTM по предсказанию цены Bitcoin (или можете взять любой другой финансовый инструмент)

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('BTC-2017min.csv')
data.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1514764740,2017-12-31 23:59:00,BTC/USD,13913.28,13913.28,13867.18,13880.00,0.591748,8213.456549
1,1514764680,2017-12-31 23:58:00,BTC/USD,13913.26,13953.83,13884.69,13953.77,1.398784,19518.309658
2,1514764620,2017-12-31 23:57:00,BTC/USD,13908.73,13913.26,13874.99,13913.26,0.775012,10782.944294
3,1514764560,2017-12-31 23:56:00,BTC/USD,13827.00,13908.69,13827.00,13859.58,0.666459,9236.841134
4,1514764500,2017-12-31 23:55:00,BTC/USD,13825.05,13825.05,13825.05,13825.05,0.065501,905.560130


In [3]:
data = pd.read_csv('BTC-2018min.csv')
data.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1546300740,2018-12-31 23:59:00,BTC/USD,3689.26,3693.30,3689.26,3693.30,9.838855,36337.841473
1,1546300680,2018-12-31 23:58:00,BTC/USD,3689.26,3691.35,3689.26,3691.35,0.560000,2067.156000
2,1546300620,2018-12-31 23:57:00,BTC/USD,3688.83,3689.26,3688.83,3689.26,0.560833,2069.059602
3,1546300560,2018-12-31 23:56:00,BTC/USD,3687.87,3689.65,3686.92,3686.92,7.610240,28058.346208
4,1546300500,2018-12-31 23:55:00,BTC/USD,3688.28,3688.85,3685.00,3688.85,7.665703,28277.629729


In [4]:
data = pd.read_csv('BTC-2019min.csv')
data.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1577836740,2019-12-31 23:59:00,BTC/USD,7167.30,7171.22,7167.30,7168.36,0.191448,1372.366393
1,1577836680,2019-12-31 23:58:00,BTC/USD,7166.89,7167.30,7161.99,7167.30,0.568868,4077.245538
2,1577836620,2019-12-31 23:57:00,BTC/USD,7164.22,7170.80,7161.65,7166.89,3.959440,28376.874525
3,1577836560,2019-12-31 23:56:00,BTC/USD,7182.49,7182.49,7170.20,7170.20,6.248892,44805.802550
4,1577836500,2019-12-31 23:55:00,BTC/USD,7175.69,7176.68,7175.69,7176.68,0.016877,121.122623


In [5]:
data = pd.read_csv('BTC-2020min.csv')
data.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1609459140,2020-12-31 23:59:00,BTC/USD,28982.44,29024.59,28969.83,28992.79,5.897679,170990.182941
1,1609459080,2020-12-31 23:58:00,BTC/USD,29044.79,29048.62,29000.12,29000.12,20.362289,590508.837815
2,1609459020,2020-12-31 23:57:00,BTC/USD,29039.53,29048.63,29023.26,29044.79,14.503728,421257.721488
3,1609458960,2020-12-31 23:56:00,BTC/USD,29052.02,29059.42,29039.28,29039.53,2.707583,78626.925850
4,1609458900,2020-12-31 23:55:00,BTC/USD,29036.10,29052.02,29025.78,29052.02,7.631439,221708.705383


In [6]:
data = pd.read_csv('BTC-2021min.csv')
data.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646106180,2022-03-01 03:43:00,BTC/USD,43046.58,43046.58,43046.58,43046.58,0.000000,0.000000
1,1646106060,2022-03-01 03:41:00,BTC/USD,43018.23,43046.59,43018.23,43046.58,0.142977,6154.673021
2,1646106000,2022-03-01 03:40:00,BTC/USD,43022.24,43022.24,43016.03,43016.03,0.009230,397.037957
3,1646105940,2022-03-01 03:39:00,BTC/USD,43035.16,43035.16,42999.44,42999.44,0.820950,35300.390268
4,1646105880,2022-03-01 03:38:00,BTC/USD,43077.82,43077.82,43049.46,43049.46,0.022210,956.143143


In [7]:
data = pd.read_csv('BTC-Daily.csv')
data.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646092800,2022-03-01 00:00:00,BTC/USD,43221.71,43626.49,43185.48,43185.48,49.006289,2.116360e+06
1,1646006400,2022-02-28 00:00:00,BTC/USD,37717.10,44256.08,37468.99,43178.98,3160.618070,1.364723e+08
2,1645920000,2022-02-27 00:00:00,BTC/USD,39146.66,39886.92,37015.74,37712.68,1701.817043,6.418008e+07
3,1645833600,2022-02-26 00:00:00,BTC/USD,39242.64,40330.99,38600.00,39146.66,912.724087,3.573010e+07
4,1645747200,2022-02-25 00:00:00,BTC/USD,38360.93,39727.97,38027.61,39231.64,2202.851827,8.642149e+07


In [8]:
data = pd.read_csv('BTC-Hourly.csv')
data.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646092800,2022-03-01 00:00:00,BTC/USD,43221.71,43626.49,43185.48,43312.27,52.056320,2.254677e+06
1,1646089200,2022-02-28 23:00:00,BTC/USD,43085.30,43364.81,42892.37,43178.98,106.816103,4.612210e+06
2,1646085600,2022-02-28 22:00:00,BTC/USD,41657.23,44256.08,41650.29,42907.32,527.540571,2.263535e+07
3,1646082000,2022-02-28 21:00:00,BTC/USD,41917.09,41917.09,41542.60,41659.53,69.751680,2.905822e+06
4,1646078400,2022-02-28 20:00:00,BTC/USD,41361.99,41971.00,41284.11,41914.97,247.151654,1.035935e+07


In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
import matplotlib.pyplot as plt

# Укажите путь к директории с данными
data_path = r'C:\Users\285\Desktop\data_science\HomeWork13 DL 4 lesson'

# Файлы с данными
files = ['BTC-2017min.csv', 'BTC-2018min.csv', 'BTC-2019min.csv','BTC-2020min.csv', 'BTC-2021min.csv']

# 1. Загрузка и объединение данных
dfs = []
for file in files:
    df = pd.read_csv(os.path.join(data_path, file))
    dfs.append(df)

# Объединение всех данных в один DataFrame
data = pd.concat(dfs)

# Преобразование столбца 'date' в datetime и установка его индексом
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

# 2. Предобработка данных
# Выберем только цену закрытия
data = data[['close']]

# Масштабируем данные для ускорения обучения
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# 3. Разделение на тренировочные и тестовые данные
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Функция для создания наборов данных с временным шагом
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Выбираем 60 минут (точек) для предсказания следующей
time_step = 60
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Изменяем форму данных для модели LSTM (samples, timesteps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 4. Построение модели BiRNN с LSTM
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50)))
model.add(Dropout(0.2))
model.add(Dense(1))

# Компиляция модели
model.compile(optimizer='adam', loss='mean_squared_error')

# 5. Обучение модели
model.fit(X_train, y_train, epochs=25, batch_size=32)

# 6. Предсказание
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Обратное масштабирование
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Восстановление данных для графиков
train_data_plot = np.empty_like(scaled_data)
train_data_plot[:, :] = np.nan
train_data_plot[time_step:len(train_predict) + time_step, :] = train_predict

test_data_plot = np.empty_like(scaled_data)
test_data_plot[:, :] = np.nan
test_data_plot[len(train_predict) + (time_step * 2) + 1:len(scaled_data) - 1, :] = test_predict

# 7. Визуализация результатов
plt.figure(figsize=(14, 5))
plt.plot(data.index, scaler.inverse_transform(scaled_data), label='Actual Prices')
plt.plot(data.index, train_data_plot, label='Train Predict', color='green')
plt.plot(data.index, test_data_plot, label='Test Predict', color='red')
plt.legend()
plt.title('Bitcoin Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.show()


C:\Users\285\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 